## $$\textrm{Samplng & Resampling}$$

## 1. 기본 설정

In [1]:
!pip install ISLP
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)

from sklearn.model_selection import train_test_split

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 687.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 3.2 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=f23e25fb3a68032c8c9656bad877faa3d86b50c08b6b0cd678356b82aeedc80e
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
  Attempting uni

In [4]:
from functools import partial
from sklearn.model_selection import (cross_validate , KFold , ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

##2. Validation and CV   

 - Auto data 로딩 및 데이터 쪼개기
 - Validation 오차 확인

In [5]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size=100, random_state=0)

hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()
print(results.summary())

X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
train_pred = results.predict(X_train)

print(np.mean((y_train - train_pred)**2))
print(np.mean((y_valid - valid_pred)**2))

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     433.9
Date:                Tue, 08 Oct 2024   Prob (F-statistic):           1.48e-59
Time:                        21:22:16   Log-Likelihood:                -879.71
No. Observations:                 292   AIC:                             1763.
Df Residuals:                     290   BIC:                             1771.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     39.8828      0.848     47.020      0.0

In [6]:
# 훈련과 검증 오차를 확인하는 함수 생성
# -------------------------------------
def evalMSE(terms, response , train , test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

- 다항모형의 차수를 바꾸면서 검증 오차를 확인 (validation error for degrees)


In [7]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

array([23.20193005, 16.99348987, 16.9506061 ])

- 교차 검증 오차 (leave-one-out)

In [8]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792922

- 다항식의 차수를 키웠을 때의 교차검증오차(leave-one-out)를 확인

In [9]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)

for i, d in enumerate(range(1,6)):
  X = np.power.outer(H, np.arange(d+1))
  M_CV = cross_validate(M, X, Y, cv=Auto.shape[0])
  cv_error[i] = np.mean(M_CV['test_score'])

cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443029, 19.03320648])

- 40 fold-CV를 통해서 차수에 대한 예측 오차를 확인함

In [11]:
cv_error = np.zeros(5)
# 40개로 쪼개는 CV
cv = KFold(n_splits=40, shuffle=True, random_state=0)
print(cv)

# use same splits for each degree for i, d in enumerate(range(1,6)):
X = np.power.outer(H, np.arange(d+1))
for i, d in enumerate(range(1,6)):
    M_CV = cross_validate(M, X, Y, cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error
print(cv_error)

KFold(n_splits=40, random_state=0, shuffle=True)
[18.93586159 18.93586159 18.93586159 18.93586159 18.93586159]


- 두 개로 쪼개서 validation error를 확인함

In [14]:
validation = ShuffleSplit(n_splits=1, test_size=196, random_state=0)
results = cross_validate(hp_model,
Auto.drop(['mpg'], axis=1), Auto['mpg'], cv =validation)
print(results)

{'fit_time': array([0.01462007]), 'score_time': array([0.0035367]), 'test_score': array([23.61661707])}


##2. Boostrap

-  $Var ( \alpha X + ( 1- \alpha)Y)$를 최소화하는 문제: 수식에 의해서 $\alpha$에 대한 해는
$$ \frac{ \sigma_Y^2 - \sigma_{XY} }{  \sigma_X^2 + \sigma_Y^2 - 2 \sigma_{XY} }$$

In [15]:
# 주어진 데이터의 인덱싱을 통해서 공분산을 얻고 해를 구함
# -------------------------------------------------------
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) / (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

In [18]:
# 위 함수를 구동
# --------------
rng = np.random.default_rng()
idx = rng.choice(100, 100, replace=True)
idx = range(Portfolio.shape[0])
alpha_func(Portfolio, idx)
# index를 랜덤하게 정해서 결과를 확인

0.57583207459283

- 붓스트랩 기법의 적용 (알파)
   - 알파에 대한 계산을 여러 개의 복원추출 샘플로 구함
   - 여기에 나온 여러 개의 알파로 (추정량) $\hat{\alpha}$의 표준편차를 추정

In [20]:
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_ , second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2) # 표준편차 계산

alpha_SE = boot_SE(alpha_func, Portfolio ,B=1000, seed=1)
alpha_SE

0.08945522198999856

- 붓스트랩 기법의 적용 (OLS)
   - 복원추출로 여러 개의 샘플을 만들어 회귀계수들을 확인  


In [21]:
def boot_OLS(model_matrix, response, D, idx):
   D_ = D.iloc[idx]     #D.iloc #
   Y_ = D_[response]
   X_ = clone(model_matrix).fit_transform(D_)
   return sm.OLS(Y_, X_).fit().params

hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')
# 함수를 구동할 때 모든 인자가 아니라 필요 인지만 받을 수 있게 함
# 앞에 두 인자는 고정시키게 됨

In [22]:
rng = np.random.default_rng(0)
kk = [hp_func(Auto, rng.choice(392, 392, replace=True)) for _ in range(10)]
print(np.array(kk))

[[39.88064456 -0.1567849 ]
 [38.73298691 -0.14699495]
 [38.31734657 -0.14442683]
 [39.91446826 -0.15782234]
 [39.43349349 -0.15072702]
 [40.36629857 -0.15912217]
 [39.62334517 -0.15449117]
 [39.0580588  -0.14952908]
 [38.66688437 -0.14521037]
 [39.64280792 -0.15555698]]


- 붓스트랩으로 얻은 표준오차와 일반 방법론으로 얻은 표준오차를 비교함

In [23]:
def boot_OLS(model_matrix, response, D, idx):
   D_ = D.loc[idx]     #D.iloc #
   Y_ = D_[response]
   X_ = clone(model_matrix).fit_transform(D_)
   return sm.OLS(Y_, X_).fit().params

hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')
hp_se = boot_SE(hp_func, Auto, B=1000, seed=0)
print(hp_se)

hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
print(model_se)

intercept     0.717654
horsepower    0.006041
dtype: float64
intercept     0.717
horsepower    0.006
Name: std err, dtype: float64


In [ ]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
print(boot_SE(quad_func, Auto, B=1000))

M = sm.OLS(Auto['mpg'], quad_model.fit_transform(Auto))
print(summarize(M.fit())['std err'])

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64
intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64
